In [2]:
import sys, os
import openai
from openai import OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

from dotenv import load_dotenv
import argparse
import json
from typing import Dict, List
import logging
import GraphDB.utils as utils
from GraphDB.LegalGraphDB import LegalGraphDB

load_dotenv(verbose=True)

# 로깅 설정 (INFO 레벨)
logging.basicConfig(level=logging.INFO)
path = os.getcwd()
root_path = os.path.dirname(path)


import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# config.json 파일 경로를 절대 경로로 설정
config_path = os.path.join(root_path, 'codes', 'configs', 'config_ra4.json')
with open(config_path, 'r') as f:
    config = json.load(f)
print(config)
dbms = LegalGraphDB(auradb=False, config=config)

# vector search Library 설치 
##### Graph Data Science -> 플러그인 

In [41]:
QUERY_1 = "증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?"
QUERY_2 = "회사채의 무보증 후순위사채 기업실사(Due Diligence)를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?"
QUERY_3 = "주식회사의 주주총회에서 주주의 의결권을 제한하는 경우에는 어떤 경우가 있나?"

embed_1 = utils.text_embed(QUERY_1, config["embedding_model"])
embed_2 = utils.text_embed(QUERY_2, config["embedding_model"])
embed_3 = utils.text_embed(QUERY_3, config["embedding_model"])

print(len(embed_1), len(embed_2), len(embed_3))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


3072 3072 3072


In [13]:
import time
from neo4j import GraphDatabase
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

# Neo4j 연결 설정
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(uri, auth=(user, password))

QUERY= "증권신고서의 효력 발생기간은?"
ANSWER  = """
증권신고서의 효력 발생 기간은 보통 접수일로부터 15일 후에 발생합니다. 그러나 금융감독원이 보완 요청을 하거나 심사가 필요할 경우 효력 발생이 연기될 수 있습니다. 아래는 구체적인 내용입니다:

기본 규정:

증권신고서는 제출 후 15일이 지나면 효력이 발생합니다.
만약 증권의 종류에 따라 법령이 달리 규정되어 있다면 해당 규정이 우선 적용됩니다.
보완 요청 시:

금융감독원이 증권신고서에 대해 보완을 요청하면, 효력 발생은 그 보완서가 제출된 날부터 다시 15일이 경과한 후로 연기됩니다.
즉시 효력 발생 요건:

일부 발행(예: 유상증자)의 경우 신고서 제출 후 15일이 아닌 즉시 효력이 발생하도록 하는 절차가 존재합니다. 이러한 경우는 발행기업의 신속한 자본 조달이 필요한 상황 등을 고려한 예외 조치입니다.
따라서 기본적으로는 15일이 표준이지만, 상황에 따라 효력 발생 시점이 달라질 수 있습니다.
"""

QUERY_EMB = utils.text_embed(QUERY, config["embedding_model"])
ANSWER_EMB = utils.text_embed(ANSWER, config["embedding_model"])

print(QUERY)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


증권신고서의 효력 발생기간은?
Total time taken: 9.77 seconds


========= Comparison of Hop-based and Vector-based Similarity Results =========
length of top_k_results:  12

####### idx: 0 #######
## VECTOR ## : Labels: Clause_01_enforcement_main, Index: 제121조제2항, Similarity: 0.6657
## HOP ## : hops : 0, Labels: Clause_01_enforcement_main, Index: 제121조제2항, Similarity: 0.6657

####### idx: 1 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제120조제1항, Similarity: 0.6530
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제120조제1항, Similarity: 0.6530

####### idx: 2 #######
## VECTOR ## : Labels: Clause_01_order_main, Index: 제12조제1항, Similarity: 0.6524
## HOP ## : hops : 0, Labels: Clause_01_order_main, Index: 제12조제1항, Similarity: 0.6524

####### idx: 3 #######
## VECTOR ## : Labels: Clause_01_order_main, Index: 제12조제4항, Similarity: 0.6448
## HOP ## : hops : 0, Labels: Clause_01_order_main, Index: 제12조제4항, Similarity: 0.6448

####### idx: 4 #######
## VECTOR ## : Labels: Clause_01_law_main

In [ ]:

# Neo4j에서 쿼리 벡터와 유사한 노드 검색
def find_similar_nodes(query_embedding_vec, driver, config):
    retrieved_nodes = []  
    top_k = config.get("top_k", 5)  # config에서 top_k 값을 가져오며 기본값을 5로 설정
    query = f"""
    WITH $embedding AS query_embedding_vec
    MATCH (n)
    WHERE ANY(label IN labels(n) WHERE label STARTS WITH 'Clause')
    WITH n, gds.similarity.cosine(n.embedding, query_embedding_vec) AS similarity
    ORDER BY similarity DESC
    LIMIT {top_k}
    RETURN n, similarity
    """
    with driver.session(database=config["database"]) as session:
        result = session.run(query, embedding=query_embedding_vec)  
        for record in result:
            node = record['n']
            node_id = node.element_id  # 노드의 고유 식별자를 사용하여 중복 방지
            similarity = record['similarity']
            labels = list(node.labels)[0] if len(node.labels) > 0 else "Unknown"  # 리스트 형태에서 첫 번째 레이블만 추출
            retrieved_nodes.append({
                'node_id': node_id,
                'index': node.get('law_index'),
                'labels': labels,
                'text': node.get('text'),
                'similarity': similarity,
                'hop': 0  # Top K 노드는 hop 0으로 표시
            })
    
    return retrieved_nodes

# Neo4j에서 유사한 노드를 검색하고 refers_to 엣지를 따라 연결된 노드를 탐색하는 함수
def find_related_nodes_with_keywords(query_embedding_vec, keywords, config, hop=1):
    
    # Top K 유사한 노드 검색
    top_k_nodes = find_similar_nodes(query_embedding_vec, driver, config)

    # Top K 노드의 element_id를 추출하여 이웃 노드 탐색 시작
    top_k_element_ids = [node['node_id'] for node in top_k_nodes]
    visited_nodes = set(top_k_element_ids)  # 중복된 노드 방문 방지

    # Top K 노드를 관련 문서 집합에 추가
    related_documents = set()
    for node in top_k_nodes:
        related_documents.add(frozenset(node.items()))  

    def recursive_search(element_ids, current_hop):
        if current_hop > hop:
            return set()  

        related_nodes = set()  
        for element_id in element_ids:
            query = """
            MATCH (n)-[:refers_to]->(neighbor)
            WHERE elementId(n) = $element_id
            RETURN neighbor
            """

            with driver.session(database=config["database"]) as session:
                result = session.run(query, element_id=element_id)
                for record in result:
                    neighbor = record['neighbor']
                    neighbor_id = neighbor.element_id  
                    
                    # 이미 방문한 노드 제외 
                    if neighbor_id in visited_nodes:
                        continue

                    visited_nodes.add(neighbor_id)
                    labels = list(neighbor.labels)[0] if len(neighbor.labels) > 0 else "Unknown"

                    # 이웃 노드의 embedding을 가져와 유사도 계산
                    neighbor_embedding = np.array(neighbor['embedding'])
                    similarity = cosine_similarity([query_embedding_vec], [neighbor_embedding])[0][0]
                    
                    # 유사도와 키워드 조건을 만족하는지 확인
                    if similarity >= config['threshold']:
                        related_node = {
                            'node_id': neighbor_id,
                            'index': neighbor.get('law_index'),
                            'labels': labels,
                            'text': neighbor.get('text'),
                            'similarity': similarity,
                            'hop': current_hop
                        }
                        related_nodes.add(frozenset(related_node.items()))  # frozenset으로 변환 후 추가하여 중복 방지
                        
                        # 재귀적으로 연결된 노드를 탐색 (hop을 올바르게 증가시켜 호출)
                        related_nodes.update(recursive_search([neighbor_id], current_hop + 1))
        return related_nodes

    # 이웃 노드를 탐색하고 유사도와 키워드 필터링 적용
    for element_id in top_k_element_ids:
        related_documents.update(recursive_search([element_id], current_hop=1))

    related_documents = [dict(doc) for doc in related_documents]  # frozenset을 dict로 변환하여 리스트로 정렬
    related_documents = sorted(related_documents, key=lambda x: x['similarity'], reverse=True)

    return top_k_nodes, related_documents

keywords = ['증권신고서', '효력 발생기간']
# Start timing
start_time = time.time()

# 단순 벡터 유사도 검색 결과와 hop 검색 결과를 얻음
top_k_results, related_docs = find_related_nodes_with_keywords(QUERY_EMB, keywords, config, hop=5)
answer_top_k_results, answer_related_docs = find_related_nodes_with_keywords(ANSWER_EMB, keywords, config, hop=5)
# hop 검색 결과의 길이를 config에 설정
config['top_k'] = len(related_docs)

# 수정된 top_k로 다시 단순 벡터 유사도 검색
top_k_results = find_similar_nodes(QUERY_EMB, driver, config)
# 수정된 top_k로 다시 단순 벡터 유사도 검색
answer_top_k_results = find_similar_nodes(ANSWER_EMB, driver, config)

# End timing
end_time = time.time()

# Calculate total time taken
total_time = end_time - start_time


print(f"Total time taken: {total_time:.2f} seconds\n")

# 인덱스별로 Top-K 검색 결과와 Hop 검색 결과를 한 번에 출력
print("\n========= Comparison of Hop-based and Vector-based Similarity Results =========")
max_length = min(len(top_k_results), len(related_docs))
print("length of top_k_results: ", len(top_k_results))

for idx in range(max_length):
    top_k_node = top_k_results[idx]
    related_doc = related_docs[idx]
    
    print(f"\n####### idx: {idx} #######")
    print(f"## VECTOR ## : Labels: {top_k_node['labels']}, Index: {top_k_node['index']}, Similarity: {top_k_node['similarity']:.4f}")
    print(f"## HOP ## : hops : {related_doc['hop']}, Labels: {related_doc['labels']}, Index: {related_doc['index']}, Similarity: {related_doc['similarity']:.4f}")


In [ ]:

with open("../data/graph/retrieval/hop-retrieve_results.json", "w") as f:
    json.dump(related_docs, f, indent=4, ensure_ascii=False)
    
# top_k_results, answer_top_k_results 를 json 형태 저장 
with open("../data/graph/retrieval/top_k_results.json", "w") as f:
    json.dump(top_k_results, f, indent=4, ensure_ascii=False)

with open("../data/graph/retrieval/answer_top_k_results.json", "w") as f:
    json.dump(answer_top_k_results, f, indent=4, ensure_ascii=False)

with open("../data/graph/retrieval/answer_related_docs.json", "w") as f:
    json.dump(answer_related_docs, f, indent=4, ensure_ascii=False)

In [9]:

vector_node_ids = set([node['node_id'] for node in top_k_results])
hop_node_ids = set([doc['node_id'] for doc in related_docs])

# 벡터 기반 검색 결과에는 있지만 hop 기반 검색 결과에는 없는 노드
only_in_vector = vector_node_ids - hop_node_ids
print(f"\n####### Node in Vector-based Only #######")
for idx, node in enumerate(top_k_results):
    if node['node_id'] in only_in_vector:
        
        print(f"idx : {idx}, Labels: {node['labels']}, Index: {node['index']}, Similarity: {node['similarity']:.4f}")


print(f"\n####### Node in Hop-based Only #######")
# hop 기반 검색 결과에는 있지만 벡터 기반 검색 결과에는 없는 노드
only_in_hop = hop_node_ids - vector_node_ids
for idx, doc in enumerate(related_docs):
    if doc['node_id'] in only_in_hop:
        
        print(f"idx : {idx}, Labels: {doc['labels']}, Index: {doc['index']}, Similarity: {doc['similarity']:.4f}")



####### Node in Vector-based Only #######
idx : 60, Labels: Clause_01_enforcement_main, Index: 제176조의15제4항, Similarity: 0.4775
idx : 62, Labels: Clause_01_enforcement_main, Index: 제92조제4항, Similarity: 0.4772
idx : 63, Labels: Clause_01_enforcement_main, Index: 제131조제4항, Similarity: 0.4754
idx : 64, Labels: Clause_01_order_main, Index: 제2조, Similarity: 0.4751
idx : 66, Labels: Clause_01_law_main, Index: 제159조제3항, Similarity: 0.4730
idx : 67, Labels: Clause_01_law_main, Index: 제112조제6항, Similarity: 0.4726
idx : 68, Labels: Clause_01_enforcement_main, Index: 제324조의9제3항, Similarity: 0.4718
idx : 69, Labels: Clause_01_law_main, Index: 제165조제4항, Similarity: 0.4717
idx : 70, Labels: Clause_01_law_main, Index: 제33조제2항, Similarity: 0.4709
idx : 71, Labels: Clause_01_enforcement_main, Index: 제369조제4항, Similarity: 0.4691
idx : 72, Labels: Clause_01_law_main, Index: 제258조제4항, Similarity: 0.4687
idx : 73, Labels: Clause_01_law_main, Index: 제254조제4항, Similarity: 0.4686
idx : 74, Labels: Clause_01_l

In [14]:

# query -> vector embedding , keyword 추출 
query = "증권신고서 제출기한 연장에 대한 법적 책임은 무엇인가요?"
query_embedding_vector = utils.text_embed(text = query, embed_model = config.get('embedding_model') )
query_keywords = utils.extract_keyword(text=query, prompt_path = config["query_keyword_prompt_path"],keyword_model = config["model"])
print(f"## query : {query}, query_keywords : {query_keywords}, len(query_embedding_vector) : {len(query_embedding_vector)}")



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## query : 증권신고서 제출기한 연장에 대한 법적 책임은 무엇인가요?, query_keywords : ['증권신고서', '제출기한 연장', '법적 책임'], len(query_embedding_vector) : 3072


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

# hop 별로  정렬하기 



# LLM based answer (No document)

In [ ]:
# LLM based answer 산출하고 retrieve 해보기



# .py 위한 재료 


In [ ]:


def main(config: Dict):
    
    parser = argparse.ArgumentParser()
    g = parser.add_argument_group("Settings")
    g.add_argument("--topK", type=int, default=10, help="Number of top related documents to retrieve")
    g.add_argument("--threshold", type=float, default=0.5, help="Threshold for relatedness score")
    g.add_argument("--output", type=str, default="C:/Users/Shic/development/GIB/legal_graph-main/data/graph/clause/retrieve/related_documents.json", help="Output file path")
    g.add_argument("--query" , type=str, default="C:/Users/Shic/development/GIB/legal_graph-main/data/graph/clause/retrieve/query.txt", help="Query file path")
    

    args = parser.parse_args()
    database = config.get("database")

    
    dbms = LegalGraphDB(auradb=False, config=config)

    # query -> vector embedding , keyword 추출 
    query_embedding_vector = utils.text_embed(config, args.query)
    query_keywords = extract_keyword(config, args.query)
    
    # 전체 node 중 query와 가장 유사한 topK개의 node_id를 반환
    retrieved_nodes = retrieve_related_documents(dbms, config , embed_vec = query_embedding_vector, keyword = query_keywords) 

    
    
    # topk 노드와 간선으로 연결된 node 중 (유사도가 threshold 이상) & (query에서 추출한 keyword를 포함하고 있으면서 ) hop1의 node_id를 반환
    # hopH까지 반복 
    # 반환된 document 에서 reranking -> 키워드 포함 / 유사도 / 홉 수 등 고려하여 가중치 부여 
    # reranking된 노드를 json으로 저장 
    # ex. {query : str , retrievedDocuments: {node_id : {keyword : [keyword1, keyword2, ...], similarity : 0.8, hop : 2}}} 


if __name__ == "__main__":
    # config.json 파일 경로를 절대 경로로 설정
    config_path = os.path.join(root_path, 'codes', 'configs', 'config.json')
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    main(config=config)
